# Porespy analysis of the 'framed' scans

At the moment only in 2D.

In [ ]:
# Load the modules we need
import platform
import os
import glob
import pathlib
import pandas
import imageio
import zarr
import numpy
import scipy
import matplotlib.pyplot as plt
from matplotlib_scalebar.scalebar import ScaleBar
import seaborn
import dask
import dask_image.imread
import skimage
from dask.distributed import Client, LocalCluster
from tqdm.auto import tqdm, trange
import porespy as ps

In [ ]:
# Set dask temporary folder
# Do this before creating a client: https://stackoverflow.com/a/62804525/323100
# We use the fast internal SSD for speed reasons
import tempfile
if 'Linux' in platform.system():
    # Check if me mounted the FastSSD, otherwise go to standard tmp file
    if os.path.exists(os.path.join(os.sep, 'media', 'habi', 'Fast_SSD')):
        tmp = os.path.join(os.sep, 'media', 'habi', 'Fast_SSD', 'tmp')
    else:
        tmp = tempfile.gettempdir()
elif 'Darwin' in platform.system():
    tmp = tempfile.gettempdir()
else:
    if 'anaklin' in platform.node():
        tmp = os.path.join('F:\\tmp')
    else:
        tmp = os.path.join('D:\\tmp')
dask.config.set({'temporary_directory': tmp})
print('Dask temporary files go to %s' % dask.config.get('temporary_directory'))

In [ ]:
# Set preferred seaborn theme
seaborn.set_theme(style='ticks',
                  context='paper')

In [ ]:
# Set figure defaults
plt.rc('image', cmap='gray', interpolation='nearest')  # Display all images in b&w and with 'nearest' interpolation
# scalefactor = 1
plt.rcParams['figure.figsize'] = (4, 4)  # Scale figures to approx. 10 cm, which is 3.93 inches
plt.rcParams['figure.dpi'] = 300

In [ ]:
# Setup scale bar defaults
plt.rcParams['scalebar.location'] = 'lower right'
plt.rcParams['scalebar.frameon'] = False
plt.rcParams['scalebar.color'] = 'white'

In [ ]:
# Load our own log file parsing code
# This is loaded as a submodule to alleviate excessive copy-pasting between *all* projects we do
# See https://github.com/habi/BrukerSkyScanLogfileRuminator for details on its inner workings
from BrukerSkyScanLogfileRuminator.parsing_functions import *

In [ ]:
if 'Win' in platform.system():
    Root = 'F:/'
else:
    Root = '/media/habi/Fast_SSD'
Path = os.path.join(Root, 'Schmid BFH Methylcellulose')
print('Our base path is %s' % Path)

In [ ]:
from dask.distributed import Client
client = Client()

In [ ]:
client


In [ ]:
# Make us a dataframe for saving all that we need
Data = pandas.DataFrame()

In [ ]:
# Get *all* log files present on disk
# Using os.walk is way faster than using recursive glob.glob
# Not sorting the found logfiles is also making it quicker
Data['LogFile'] = [os.path.join(root, name)
                   for root, dirs, files in os.walk(Path)
                   for name in files
                   if name.endswith((".log"))]

In [ ]:
# Get all folders
Data['Folder'] = [os.path.dirname(f) for f in Data['LogFile']]
Data['FolderShort'] = [folder[len(Root) + 1:] for folder in Data['Folder']]

In [ ]:
Data.sample(n=5)

In [ ]:
# Get rid of all the logfiles from all the folders that might be on disk but that we don't want to load the data from
for c, row in Data.iterrows():
    if 'proj' in os.path.split(row.Folder)[-1]:  # drop all projections folders
        Data.drop([c], inplace=True)
    if os.path.split(row.Folder)[-1] == 'PR':  # drop all phase retrieval folders for the moment
        Data.drop([c], inplace=True)        
    elif 'rectmp.log' in row.LogFile:  # drop temporary log files of samples currently being reconstructed
        Data.drop([c], inplace=True)
# Reset dataframe to something that we would get if we only would have loaded the 'rec' files
Data = Data.reset_index(drop=True)

In [ ]:
# Generate us some meaningful colums in the dataframe
Data['Sample'] = [('-').join([pathlib.Path(log).parts[-4], pathlib.Path(log).parts[-3]]) for log in Data['LogFile']]
Data['Scan'] = [os.path.basename(os.path.dirname(log)) for log in Data['LogFile']]

In [ ]:
Data.Sample.unique()

In [ ]:
# Load the file names of all the reconstructions of all the scans
Data['Filenames Reconstructions'] = [sorted(glob.glob(os.path.join(f, '*rec0*.png'))) for f in Data['Folder']]
# How many reconstructions do we have?
Data['Number of reconstructions'] = [len(r) for r in Data['Filenames Reconstructions']]

In [ ]:
# Drop samples which have either not been reconstructed yet or of which we deleted the reconstructions with
# `find . -name "*rec*.png" -type f -mtime +333 -delete`
# Based on https://stackoverflow.com/a/13851602
# for c,row in Data.iterrows():
#     if not row['Number of reconstructions']:
#         print('%s contains no PNG files, we might be currently reconstructing it' % row.Folder)
Data = Data[Data['Number of reconstructions'] > 0]
# Reset the dataframe count/index for easier indexing afterwards
Data.reset_index(drop=True, inplace=True)
print('We have %s folders with reconstructions' % (len(Data)))

In [ ]:
# Get parameters to doublecheck from logfiles
Data['Voxelsize'] = [pixelsize(log) for log in Data['LogFile']]
Data['Camera'] = [camera(log) for log in Data['LogFile']]
Data['Filter'] = [whichfilter(log) for log in Data['LogFile']]
Data['Exposuretime'] = [exposuretime(log) for log in Data['LogFile']]
Data['Scanner'] = [scanner(log) for log in Data['LogFile']]
Data['Averaging'] = [averaging(log) for log in Data['LogFile']]
Data['ProjectionSize'] = [projection_size(log) for log in Data['LogFile']]
Data['RotationStep'] = [rotationstep(log) for log in Data['LogFile']]
Data['Grayvalue'] = [reconstruction_grayvalue(log) for log in Data['LogFile']]
Data['RingartefactCorrection'] = [ringremoval(log) for log in Data['LogFile']]
Data['BeamHardeningCorrection'] = [beamhardening(log) for log in Data['LogFile']]
Data['DefectPixelMasking'] = [defectpixelmasking(log) for log in Data['LogFile']]
Data['Scan date'] = [scandate(log) for log in Data['LogFile']]

In [ ]:
# Get rid of all the scans except 'Framed'
for c, row in Data.iterrows():
    if 'Blobs' in row.LogFile or 'Chunks' in row.LogFile or 'Kreidegrund' in row.LogFile:
        Data.drop([c], inplace=True)
# Reset dataframe to something that we would get if we only would have loaded the 'rec' files
Data = Data.reset_index(drop=True)

In [ ]:
Data[['Sample', 'Scan']]

In [ ]:
Data.Sample.unique()

In [ ]:
# # Load all reconstructions DASK arrays
# Reconstructions = [dask_image.imread.imread(os.path.join(folder,'*rec*.png')) for folder in Data['Folder']]
# Load all reconstructions into ephemereal DASK arrays, with a nice progress bar...
Reconstructions = [None] * len(Data)
for c, row in tqdm(Data.iterrows(),
                   desc='Loading reconstructions',
                   total=len(Data)):
    Reconstructions[c] = dask_image.imread.imread(os.path.join(row['Folder'], '*rec*.png'))[:,:,:,0]

In [ ]:
# If we have more than one sample load only the selected one
whichscan = 0
print('Loading reconstructions for %s' % Data['Sample'][whichscan])

In [ ]:
Reconstructions[whichscan]

In [ ]:
# Load test image from the midle of the stack
# Just the middle slice of the stack
image = Reconstructions[whichscan][Reconstructions[whichscan].shape[0]//2].compute()
# The middle slice in x
# inputimage = Reconstructions[whichscan][:,Reconstructions[whichscan].shape[1]//2,:].compute()
# The middle slice in 
# inputimage = Reconstructions[whichscan][:,:,Reconstructions[whichscan].shape[2]//2].compute()

In [ ]:
# Show test image
plt.imshow(skimage.exposure.equalize_adapthist(image))
plt.title('Input image, %s x %s px' % (image.shape[0], image.shape[1]))
plt.gca().add_artist(ScaleBar(Data['Voxelsize'][whichscan],'um'))
plt.axis('off')
plt.show()

In [ ]:
# Top and bottom of the image is empty, detect the region, based on its horizontal MIP
grayvalue_vertical = numpy.max(image, axis=1)
# Find region from left and right based on the Otsu threshold
threshold_v = skimage.filters.threshold_otsu(grayvalue_vertical)
buffer_v = 0
top = numpy.where(grayvalue_vertical > threshold_v)[0][0] - buffer_v
bottom = numpy.where(grayvalue_vertical > threshold_v)[0][-1] + buffer_v

In [ ]:
# Left and right of the image might be cut, too
grayvalue_horizontal = numpy.max(image, axis=0)
threshold_h = skimage.filters.threshold_otsu(grayvalue_horizontal)
buffer_h = 0
left = numpy.where(grayvalue_horizontal > threshold_h)[0][0] - buffer_h
right = numpy.where(grayvalue_horizontal > threshold_h)[0][-1] + buffer_h

In [ ]:
len(grayvalue_horizontal)

In [ ]:
# Top/Bottom
plt.subplot(121)
plt.plot(grayvalue_vertical, range(len(grayvalue_vertical)))
plt.axvline(threshold_v)
plt.axhline(top, color='red', linestyle='--', label='top@%s' % top)
plt.axhline(top + buffer_v, color='red', linestyle='--', alpha=0.5, label='top-buffer@%s' % str(top + buffer_v))
plt.axhline(bottom - buffer_v, color='blue', linestyle='--', alpha=0.5, label='bottom-buffer@%s' % str(bottom- buffer_v))
plt.axhline(bottom, color='blue', linestyle='--', label='bottom@%s' % bottom)
plt.xlim(0, 255)
plt.legend()
plt.title('Vertical gray value MIP')
seaborn.despine()
plt.subplot(122)
plt.plot(grayvalue_horizontal)
plt.axhline(threshold_h)
plt.axvline(left, color='red', linestyle='--', label='left@%s' % left)
plt.axvline(left + buffer_h, color='red', linestyle='--', alpha=0.5, label='left+buffer@%s' % str(left+buffer_h))
plt.axvline(right - buffer_h, color='blue', linestyle='--', alpha=0.5, label='right-buffer@%s' % str(right-buffer_h))
plt.axvline(right, color='blue', linestyle='--', label='right@%s' % right)
plt.ylim(0, 255)
plt.legend()
plt.title('Horizontal gray value sum')
seaborn.despine()
plt.suptitle(os.path.basename(Data['Filenames Reconstructions'][whichscan][len(Data['Filenames Reconstructions'][whichscan])//2]))
plt.show()

In [ ]:
# Crop top/bottom (calculated as left/right above)
plt.subplot(211)
plt.imshow(image)
plt.plot(grayvalue_vertical, range(len(grayvalue_vertical)), label='Vertical gray value profile')
plt.axhline(top, ls='dashed', c='w', alpha=0.618, label='Cut top@%s' % top)
plt.axhline(bottom, ls='dashed', c='w', alpha=0.618, label='Cut bottom@%s' % bottom)
plt.plot(grayvalue_horizontal)
plt.axvline(left, ls='dotted', c='w', alpha=0.618, label='Cut left@%s' % left)
plt.axvline(right, ls='dotted', c='w', alpha=0.618, label='Cut right@%s' % right)
plt.legend()
plt.gca().add_artist(ScaleBar(Data['Voxelsize'][whichscan],'um'))
plt.axis('off')
plt.subplot(212)
croppedimage = image[top:bottom, left:right]
plt.axis('off')
plt.imshow(skimage.exposure.equalize_adapthist(croppedimage))
plt.gca().add_artist(ScaleBar(Data['Voxelsize'][whichscan],'um'))
plt.show()

In [ ]:
def crop_image(image, buffer_h=1, buffer_v=1, verbose=False):
    '''Crop input image to smallest left:right,top:bottom rectangle, based on horizontal and vertical MIP'''
    # Top and bottom of the image is empty, detect the region, based on its horizontal MIP
    grayvalue_vertical = numpy.max(image, axis=1)
    threshold_v = skimage.filters.threshold_otsu(grayvalue_vertical)
    top = numpy.where(grayvalue_vertical > threshold_v)[0][0] - buffer_v
    bottom = numpy.where(grayvalue_vertical > threshold_v)[0][-1] + buffer_v
    # Left and right of the image might be cut, too
    grayvalue_horizontal = numpy.max(image, axis=0)
    threshold_h = skimage.filters.threshold_otsu(grayvalue_horizontal)
    left = numpy.where(grayvalue_horizontal > threshold_h)[0][0] - buffer_h
    right = numpy.where(grayvalue_horizontal > threshold_h)[0][-1] + buffer_h
    croppedimage = image[top:bottom, left:right]
    if verbose:
        # Top/Bottom
        plt.subplot(221)
        plt.plot(grayvalue_vertical, range(len(grayvalue_vertical)))
        plt.axvline(threshold_v)
        plt.axhline(top, color='red', linestyle='--', label='top@%s' % top)
        plt.axhline(top + buffer_v, color='red', linestyle='--', alpha=0.5, label='top-buffer@%s' % str(top + buffer_v))
        plt.axhline(bottom - buffer_v, color='blue', linestyle='--', alpha=0.5, label='bottom-buffer@%s' % str(bottom- buffer_v))
        plt.axhline(bottom, color='blue', linestyle='--', label='bottom@%s' % bottom)
        plt.xlim(0, 255)
        plt.legend()
        plt.title('Vertical gray value MIP')
        seaborn.despine()
        plt.subplot(222)
        plt.plot(grayvalue_horizontal)
        plt.axhline(threshold_h)
        plt.axvline(left, color='red', linestyle='--', label='left@%s' % left)
        plt.axvline(left + buffer_h, color='red', linestyle='--', alpha=0.5, label='left+buffer@%s' % str(left+buffer_h))
        plt.axvline(right - buffer_h, color='blue', linestyle='--', alpha=0.5, label='right-buffer@%s' % str(right-buffer_h))
        plt.axvline(right, color='blue', linestyle='--', label='right@%s' % right)
        plt.ylim(0, 255)
        plt.legend()
        plt.title('Horizontal gray value sum')
        seaborn.despine()
        plt.subplot(223)
        plt.imshow(image)
        plt.title('%s x %s px' % (image.shape[0], image.shape[1]))
        plt.plot(grayvalue_vertical, range(len(grayvalue_vertical)), label='Vertical gray value profile')
        plt.axhline(top, ls='dashed', c='w', alpha=0.618, label='Cut top@%s' % top)
        plt.axhline(bottom, ls='dashed', c='w', alpha=0.618, label='Cut bottom@%s' % bottom)
        plt.plot(grayvalue_horizontal)
        plt.axvline(left, ls='dotted', c='w', alpha=0.618, label='Cut left@%s' % left)
        plt.axvline(right, ls='dotted', c='w', alpha=0.618, label='Cut right@%s' % right)
        plt.legend()
        plt.gca().add_artist(ScaleBar(Data['Voxelsize'][whichscan],'um'))
        plt.axis('off')
        plt.subplot(224)
        plt.axis('off')
        plt.imshow(skimage.exposure.equalize_adapthist(croppedimage))
        plt.title('%s x %s px' % (croppedimage.shape[0], croppedimage.shape[1]))
        plt.gca().add_artist(ScaleBar(Data['Voxelsize'][whichscan],'um'))
        plt.show()
    return(croppedimage)

In [ ]:
threshold_iso = skimage.filters.threshold_isodata(croppedimage)

In [ ]:
binarizedimage = croppedimage < threshold_iso  # porespy expects 'True' for features of interest, so we true stuff smaller than the threshold, e.g the air

In [ ]:
plt.subplot(211)
plt.imshow(croppedimage)
plt.title('Center of original slice, %s x %s px' % (croppedimage.shape[0], croppedimage.shape[1]))
plt.gca().add_artist(ScaleBar(Data['Voxelsize'][whichscan],'um'))
plt.axis('off')
plt.subplot(212)
plt.imshow(~binarizedimage) # Invert for displaying
plt.title('Binarized image, %s x %s px' % (binarizedimage.shape[0], binarizedimage.shape[1]))
plt.gca().add_artist(ScaleBar(Data['Voxelsize'][whichscan],'um'))
plt.axis('off')
plt.show()

In [ ]:
lt = ps.filters.local_thickness(binarizedimage)
pt = ps.filters.porosimetry(binarizedimage)

In [ ]:
plt.subplot(121)
plt.imshow(lt)
plt.subplot(122)
plt.imshow(pt)
plt.show()

In [ ]:
# Visualize difference
plt.imshow(lt-pt)
plt.show()

Local thickness according to https://porespy.org/examples/filters/reference/local_thickness.html

In [ ]:
# Calculate local thickness
localthickness = ps.filters.local_thickness(binarizedimage, sizes=255)

In [ ]:
# Show local thickness over binarized image
plt.imshow(localthickness/binarizedimage, cmap='viridis')
plt.gca().add_artist(ScaleBar(Data['Voxelsize'][whichscan],'um'))
plt.axis('off')
plt.title('Local thickness overlaid over original')
plt.show()

Pore size distribution as per https://porespy.org/examples/metrics/reference/pore_size_distribution.html

In [ ]:
psd = ps.metrics.pore_size_distribution(localthickness,
                                        bins=100,
                                        log=False,
                                        voxel_size=Data['Voxelsize'][whichscan])
print(psd)

In [ ]:
# Plot probability density function, based on https://porespy.org/examples/metrics/reference/pore_size_distribution.html
plt.bar(x=psd.R, height=psd.pdf, width=psd.bin_widths)
plt.xlabel('Pore radius [um]')
plt.ylabel('Probability density function')
plt.xlim([0, 500])
seaborn.despine()


Now that we have "singled out" all the different steps, we can put them together into a function.

In [ ]:
# Put all together into a "function" that we can call for each reconstruction
def analyze_reconstruction(whichscan, whichreconstruction, verbose=True):
    # Load the requested reconstruction
    inputimage = Reconstructions[whichscan][whichreconstruction].compute()
    if verbose:
        print('Analyzing reconstruction %s of sample %s (%s)' % (whichreconstruction,
                                                                 Data['Sample'][whichscan],
                                                                 Data['Filenames Reconstructions'][whichscan][whichreconstruction][len(Root) + 1:]))
        plt.imshow(inputimage)
        plt.title('%s, %s x %s px @ %0.2f um' % (Data['Filenames Reconstructions'][whichscan][whichreconstruction][len(Root) + 1:],
                                                 inputimage.shape[0], inputimage.shape[1],
                                                 Data['Voxelsize'][whichscan]))
        plt.gca().add_artist(ScaleBar(Data['Voxelsize'][whichscan],'um'))
        plt.axis('off')
        plt.show()
    # Crop it
    croppedimage = crop_image(inputimage, verbose=verbose)
    # Threshold with isodata, which - according to ChatGPT - is best for gradual transitions and less clearly defined histogram modes.
    threshold = skimage.filters.threshold_isodata(croppedimage)
    if verbose:
        # Display gray value histogram of image
        histogram = plt.hist(croppedimage.ravel(),
                            bins='doane',
                            histtype='step',
                            log=True,
                            label='Histogram',
                            color=seaborn.color_palette()[0])
        plt.axvline(threshold, label='Threshold@%s' % threshold, c=seaborn.color_palette()[1])
        plt.xlim([0, 255])
        plt.legend()
        plt.title('Logarithmic grayvalue histogram with %s bins' % len(histogram[1]))
        seaborn.despine()
        plt.show()
    binarizedimage = croppedimage < threshold  # porespy expects 'True' for features of interest, so we true stuff smaller than the threshold, e.g the air
    if verbose:
        plt.subplot(211)
        plt.imshow(croppedimage)
        plt.title('Original, cropped, %s x %s px' % (croppedimage.shape[0],
                                                     croppedimage.shape[1]))
        plt.gca().add_artist(ScaleBar(Data['Voxelsize'][whichscan],'um'))
        plt.axis('off')
        plt.subplot(212)
        plt.imshow(~binarizedimage) # Invert for display
        plt.title('Original, cropped and binarized @ %s, %s x %s px' % (threshold,
                                                                        binarizedimage.shape[0],
                                                                        binarizedimage.shape[1]))
        plt.gca().add_artist(ScaleBar(Data['Voxelsize'][whichscan],'um'))
        plt.axis('off')
        plt.show()
    # Calculate local thickness according to https://porespy.org/examples/filters/reference/local_thickness.html
    localthickness = ps.filters.local_thickness(binarizedimage,
                                                sizes=100, 
                                                )
    if verbose:
        # Show local thickness over binarized image
        plt.imshow(localthickness/binarizedimage, cmap='viridis')
        plt.gca().add_artist(ScaleBar(Data['Voxelsize'][whichscan],'um'))
        plt.axis('off')
        plt.title('Local thickness overlaid over binarized image')
        plt.show()
    # Calculate pore size distribution, as per https://porespy.org/examples/metrics/reference/pore_size_distribution.html
    psd = ps.metrics.pore_size_distribution(localthickness,
                                            bins=50,
                                            log=False,
                                            voxel_size=Data['Voxelsize'][whichscan])
    if verbose:
        print(psd)
    # Visualize results
    # Plot probability density function, based on https://porespy.org/examples/metrics/reference/pore_size_distribution.html
    plt.figure()
    plt.bar(x=psd.R, height=psd.pdf, width=psd.bin_widths)
    plt.xlabel('Pore radius [um]')
    plt.ylabel('Probability density function')
    plt.title('Pore size distribution of %s' % (Data['Filenames Reconstructions'][whichscan][whichreconstruction][len(Root) + 1:]))
    plt.xlim([0, 700])
    plt.ylim([0, 0.015])
    seaborn.despine()    
    os.makedirs(os.path.join(os.path.dirname(Data['Folder'][whichscan]), 'psd'), exist_ok=True)
    plt.savefig(os.path.join(os.path.dirname(Data['Folder'][whichscan]), 'psd', 'psd_%s' % os.path.basename(Data['Filenames Reconstructions'][whichscan][whichreconstruction])))
    if verbose:
        plt.show()
    plt.close()
    return()

In [ ]:
# results = ps.metrics.pore_size_distribution(localthickness,
#                                             bins=2**8,
#                                             log=False,
#                                             voxel_size=Data['Voxelsize'][whichscan])

In [ ]:
# print(results)

In [ ]:
# plt.figure(figsize=(8, 5))
# plt.plot(results['R'], results['pdf'], label='PDF', color='navy')
# # plt.plot(results['R'], results['cdf'], label='CDF', color='darkorange')
# plt.xlabel('Pore Radius (units of voxel size)')
# plt.ylabel('Frequency / Cumulative')
# plt.title('Pore Size Distribution')
# plt.legend()
# plt.grid(True)
# plt.tight_layout()
# plt.show()

In [ ]:
# analyze_reconstruction(0, 1001, verbose=True)

In [ ]:
# Randomly sample N reconstructions
for i in tqdm(numpy.random.sample(777)):
    analyze_reconstruction(0,
                           int(i*len(Data['Filenames Reconstructions'][whichscan])),
                           verbose=False)

In [ ]:
# whichscan = 0
# for i in trange(len(Data['Filenames Reconstructions'][whichscan]),
#                    desc='Analyzing reconstructions',
#                    total=len(Data['Filenames Reconstructions'][whichscan])):
#     if not i % 222:
#         analyze_reconstruction(whichscan, i, verbose=False)